In [1]:
import pandas as pd
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

In [71]:
train_df = pd.read_csv('/home/roland/Projects/JP_citation_classification/dynamic_citation_extraction_original/data/fixed_context_sdp_act/train.txt', sep="\t", engine="python", dtype=object)
test_df = pd.read_csv('/home/roland/Projects/JP_citation_classification/dynamic_citation_extraction_original/data/fixed_context_sdp_act/test.txt', sep="\t", engine="python", dtype=object)
val_df = pd.read_csv('/home/roland/Projects/JP_citation_classification/dynamic_citation_extraction_original/data/fixed_context_sdp_act/valid.txt', sep="\t", engine="python", dtype=object)

In [64]:
add_data = pd.read_csv('/home/roland/Projects/JP_citation_classification/feature_scraping/data/enriched_data/6_diff_in_publication_date.csv')

In [65]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   unique_id                2500 non-null   object
 1   core_id                  2500 non-null   object
 2   citing_title             2500 non-null   object
 3   citing_author            2500 non-null   object
 4   cited_title              2500 non-null   object
 5   cited_author             2500 non-null   object
 6   cited_abstract           2242 non-null   object
 7   citation_context         2500 non-null   object
 8   cite_context_paragraph   2500 non-null   object
 9   citation_class_label     2500 non-null   object
 10  cite_context_-1_sent     2500 non-null   object
 11  cite_context_sent_+1     2500 non-null   object
 12  cite_context_-1_sent_+1  2500 non-null   object
 13  cite_context_-2_sent     2500 non-null   object
 14  cite_context_sent_+2     2500 non-null  

In [66]:
extra_features = ['unique_id', 'cite_pos_in_sent', 'self_citation', 'direct_citations', 'cited_publication_date', 'cited_publication_info', 'sent_pos_in_article']

In [67]:
train_df = pd.merge(left = train_df, right = add_data[extra_features], on = 'unique_id', how = 'left').dropna()
test_df = pd.merge(left = test_df, right = add_data[extra_features], on = 'unique_id', how = 'left')#.dropna()
val_df = pd.merge(left = val_df, right = add_data[extra_features], on = 'unique_id', how = 'left')

<ipython-input-77-59921c4cb84e>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  check = pd.concat(test_df.unique_id, train_df.unique_id)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [73]:
add_data.head()

,unique_id,core_id,citation_offset,total_doc_length,section_info,citing_title,citing_author,citing_publication_info,citing_abstract,cited_title,...,direct_citations,co_mentions,citation_class_label,citation_influence_label,cite_pos_in_sent,sent_pos_in_article,article_id,citing_publication_dat,reference_key,diff_publication_date
0,CC1,81153632,2302.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,The English Village Community Examined in its ...,...,1.0,1.0,5,0,0.398827,0.038544,96,2013,NaN,NaN
1,CC2,81153632,23562.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,Die italienischen Begründer der Wiener Donauka...,...,1.0,1.0,0,1,0.929245,0.394515,96,2013,96_b60,36.0
2,CC3,81153632,22736.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,Wien im Bild historischer Karten Die Entwicklu...,...,1.0,2.0,4,1,0.912281,0.380684,96,2013,96_b48,9.0
3,CC4,81153632,46410.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,The hydraulic geometry of stream channels and ...,...,1.0,0.0,0,0,0.484615,0.777075,96,2013,96_b37,60.0
4,CC5,81153632,4191.0,59724,"{""Introduction"": 2011, ""Synthesis"": 54734}","Two steps back, one step forward: reconstructi...",Severin Hohensinner,2013,As part of an interdisciplinary project on the...,From memory to written record,...,1.0,0.0,0,0,0.888060,0.070173,96,2013,96_b10,20.0


In [53]:
features = ['cite_pos_in_sent', 'cited_publication_date', 'direct_citations', 'self_citation', 'sent_pos_in_article']
target_column = 'citation_class_label'

In [43]:
X_train = train_df[features].astype(float)
y_train = train_df[target_column].astype(float)
X_test = test_df[features].astype(int)
y_test = test_df[target_column].astype(int)

In [51]:
len(y_train)

1928

# XGboost

In [44]:
# Create an instance of the classifier
model = XGBClassifier()

# Train it on the training set.
model.fit(X_train, y_train)

# Use the trained model to predict the labels for the test set.
predictions = model.predict(X_test)

# Calculate the F1 score.
f1 = f1_score(y_true = y_test, y_pred = predictions, average='weighted')

print('Using XGBoost on non-text features...')
print('\nF1: %.3f' % f1)

ValueError: attempt to get argmax of an empty sequence

In [49]:
y_test.isna().sum()

0

In [33]:
y_test

0      5
1      0
2      4
3      0
4      0
      ..
995    1
996    3
997    3
998    3
999    3
Name: citation_class_label, Length: 1000, dtype: int64